# Progressive GAN

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Generator
* TODO: Change UpsamplingBilinear to interpolate()
* TODO: Move to inner class

In [127]:
class GeneratorBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.upsample = nn.UpsamplingBilinear2d(scale_factor=2)
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        
    def forward(self, x):
        _, x = self.forward_with_img(x)
        
        return x
    
    def forward_with_img(self, x):
        img = self.upsample(x)
        x = F.leaky_relu(self.conv1(img))
        x = F.leaky_relu(self.conv2(x))
        
        return img, x

In [171]:
class Generator(nn.Module):
    def __init__(self, channels=512):
        super().__init__()
        self.channels = channels
        self.trained_blocks = nn.ModuleList()
        self.new_block = self.create_initial_layer(channels)
        self.rgb = self.to_rgb(channels)
        
    def to_rgb(self, in_channels):
        return nn.Conv2d(in_channels, 3, 1, padding=0)
    
    def create_initial_layer(self, channels):
        module = nn.Sequential(
            nn.Conv2d(channels, channels, 4, padding=3),
            nn.LeakyReLU(),
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.LeakyReLU())
        
        return module
    
    def append_layer(self, channels):
        self.trained_blocks.append(self.new_block)
        self.new_block = GeneratorBlock(self.channels, channels)
        self.rgb = self.to_rgb(channels)
        self.channels = channels
        
    def forward(self, x, alpha=1.0):
        for block in self.trained_blocks:
            x = block(x)
            
        if alpha < 1.0:
            img , x = self.new_block.forward_with_img(x)
            img = self.rgb(img)
            x = self.rgb(x)
            
            x = img * (1 - alpha) + img * alpha
        else:
            x = self.new_block(x)
            x = self.rgb(x)
        
        return x

# Disciminator

In [172]:
class DiscriminatorBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.downsample = nn.UpsamplingBilinear2d(scale_factor=0.5)
        
    def forward(self, x, alpha=0):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.downsample(x)
        
        return x

In [173]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 1, padding=0)
        
        self.block1 = DiscriminatorBlock(16, 32)
        self.block2 = DiscriminatorBlock(32, 64)
        self.block3 = DiscriminatorBlock(64, 128)
        
        self.out = nn.Conv2d(128, 128, 4, padding=0)
        self.linear = nn.Linear(128, 1)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.out(x)
        x = self.linear(x.view(-1, 128))
        
        return x

In [182]:
g = Generator()
d = Discriminator()

In [194]:
z = torch.randn(2, 512, 1, 1)
gen_img = g(z)

In [195]:
print(gen_img.shape)

torch.Size([2, 3, 32, 32])


In [193]:
g.append_layer(128)

In [45]:
out = d(gen_img)

In [46]:
print(out.shape)

torch.Size([2, 1])
